In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Import Necessary Libraries

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as plt
import tensorflow as tf
import nltk
from tensorflow.keras.preprocessing import text, sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D, MaxPooling1D
from sklearn.model_selection import train_test_split
print(tf.__version__)

In [ ]:
MIXED_PRECISION = True
XLA_ACCELERATE  = False # Didn't work; Dunno Why!

GPUS = tf.config.experimental.list_physical_devices('GPU')
if GPUS:
    try:
        for GPU in GPUS:
            tf.config.experimental.set_memory_growth(GPU, True)
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(GPUS), "Physical GPUs,", len(logical_gpus), "Logical GPUs") 
    except RuntimeError as  RE:
        print(RE)

if MIXED_PRECISION:
    policy = tf.keras.mixed_precision.experimental.Policy('mixed_float16')
    tf.keras.mixed_precision.experimental.set_policy(policy)
    print('Mixed precision enabled')

if XLA_ACCELERATE:
    tf.config.optimizer.set_jit(True)
    print('Accelerated Linear Algebra enabled')
    
strategy = tf.distribute.get_strategy()
REPLICAS = strategy.num_replicas_in_sync
print(f'REPLICAS: {REPLICAS}') 
print("Tensorflow version " + tf.__version__)

In [ ]:
train= pd.read_csv('../input/jigsaw-toxic-comment-classification-challenge/train.csv.zip')
test= pd.read_csv('../input/jigsaw-toxic-comment-classification-challenge/test.csv.zip')
test_labels= pd.read_csv('../input/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip')
sample_submission= pd.read_csv('../input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip')

In [ ]:
print("train datas")
print(train.head())


print('----------------------------')
print("Test data")
print(test.head())


print('----------------------------')
print('Test Labels')
print(test_labels.head())

In [ ]:
train.info()

In [ ]:
sample_submission.head()

In [ ]:
toxic_text= train.loc[train['toxic']==1]
toxic_text

## Lets See word Density

In [ ]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
comments = train['comment_text'].loc[train['toxic']].values
wordcloud = WordCloud(
    width = 640,
    height = 640,
    background_color = 'black',
    stopwords = STOPWORDS).generate(str(comments))
fig = plt.figure(
    figsize = (12, 8),
    facecolor = 'k',
    edgecolor = 'k')
plt.imshow(wordcloud, interpolation = 'bilinear')
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()

In [ ]:

comments = test['comment_text'].values
wordcloud = WordCloud(
    width = 640,
    height = 640,
    background_color = 'black',
    stopwords = STOPWORDS).generate(str(comments))
fig = plt.figure(
    figsize = (12, 8),
    facecolor = 'k',
    edgecolor = 'k')
plt.imshow(wordcloud, interpolation = 'bilinear')
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()

In [ ]:
values= train['comment_text'].values
#sentences= nltk.tokenize.sent_tokenize(values)
for sentence in values:
    sentences= nltk.tokenize.sent_tokenize(sentence)
    for text in sentences:
        print(text)


## Lets see the WordCloud

In [ ]:
toxic_texts= train['comment_text'].loc[train['toxic']==1].values
line=[]
for sentence in toxic_texts:
    sentences= nltk.tokenize.sent_tokenize(sentence)
    for text in sentences:
        print(text)
        line.append(text)


In [ ]:
wordcloud = WordCloud(
    width = 640,
    height = 640,
    background_color = 'black',
    stopwords = STOPWORDS).generate(str(line))
fig = plt.figure(
    figsize = (12, 8),
    facecolor = 'k',
    edgecolor = 'k')
plt.imshow(wordcloud, interpolation = 'bilinear')
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()

In [ ]:
print(train.toxic.value_counts())
sns.countplot(train.toxic)

In [ ]:
print(test_labels.toxic.value_counts())
sns.countplot(test_labels.toxic)

In [ ]:
train_data=train[['comment_text','toxic' ]]

In [ ]:
def get_ax(rows=1, cols=2, size=7):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Adjust the size attribute to control how big to render images
    """
    fig, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return fig, ax

In [ ]:
fig, ax = get_ax()

sns.distplot(train_data[train_data['toxic']==0]['comment_text'].str.len(), axlabel="Non Toxic", ax=ax[0])
sns.distplot(train_data[train_data['toxic']==0]['comment_text'].str.split().str.len(), axlabel="Non Toxic", ax=ax[1])

fig.show()

In [ ]:
fig, ax = get_ax()

sns.distplot(train_data[train_data['toxic']==1]['comment_text'].str.len(), axlabel="Toxic", ax=ax[0])
sns.distplot(train_data[train_data['toxic']==1]['comment_text'].str.split().str.len(), axlabel="Toxic", ax=ax[1])

fig.show()

In [ ]:
label_columns = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
# Assigned to count all clean comments
value_counts_0 = []
# Assigned to count all offensive comments
value_counts_1 = []

for col in label_columns:
  value_counts_0.append(train[col].value_counts()[0])
  value_counts_1.append(train[col].value_counts()[1])

print("Number of clean comments:",value_counts_0)
print("Number of offensive comments:",value_counts_1)

In [ ]:
def plot_toxicity(labels,count,ylabel,xlabel,subtitle):
    fig = plt.figure(figsize = (10, 5)) 

    # creating the bar plot 
    plt.bar(labels,count, color=['#422680','#341671','#280659','#660F56','#AE2D67','#F54952'],  width = 0.5) 

    plt.xlabel(xlabel,fontweight ='bold',fontname='Monsterrat') 
    plt.ylabel(ylabel,fontweight ='bold',fontname='Monsterrat') 
    plt.title(subtitle,fontweight ='bold',fontname='Comic Sans MS') 

    for x,y in zip(labels,count):

        label = "{:}".format(y)

        plt.annotate(label, # this is the text
                     (x,y), # this is the point to label
                     textcoords="offset points", # how to position the text
                     xytext=(0,3), # distance from text to points (x,y)
                     ha='center') # horizontal alignment can be left, right or c
    
    plt.show() 

plot_toxicity(label_columns,value_counts_0,"No. of occurences","Type of toxicity","Clean Comments")
plot_toxicity(label_columns,value_counts_1,"No. of occurences","Type of toxicity","Toxicity Distribution")

In [ ]:
offensive= train.iloc[:, 2:].sum()
row_sum= train.iloc[:, 2:].sum(axis=1)

train['clean']= (row_sum ==0)
print('Total Comments:', len(train))
print('Total clean comments:', train['clean'].sum())
print('Offensive texts:', offensive.sum())

In [ ]:
clean_lines= train[train['clean']==True]
offensive_lines= train[train['clean']==False]

In [ ]:
def cal_len(data):
    return len(data)

#Create generic plotter with Seaborn
def plot_count(count_ones,count_zeros,title_1,title_2,subtitle):
    fig,(ax1,ax2)=plt.subplots(1,2,figsize=(15,10))
    sns.distplot(count_zeros,ax=ax1,color='#422680')
    ax1.set_title(title_1)
    sns.distplot(count_ones,ax=ax2,color='#F54952')
    ax2.set_title(title_2)
    fig.suptitle(subtitle)
    plt.show()    


count_clean_words=clean_lines['comment_text'].str.split().apply(lambda z:cal_len(z))
count_offensive_words=offensive_lines['comment_text'].str.split().apply(lambda z:cal_len(z))
plot_count(count_clean_words,count_offensive_words,"Clean Words","Offensive words","Comments Word Analysis")

In [ ]:
import string
count_clean_punctuations=clean_lines['comment_text'].apply(lambda z: len([c for c in str(z) if c in string.punctuation]))
count_offensive_punctuations=offensive_lines['comment_text'].apply(lambda z:len([c for c in str(z) if c in string.punctuation]))
plot_count(count_clean_punctuations,count_offensive_punctuations,"Clean Comments Punctuations","Offensive Comments Punctuations","Comments Word Punctuation Analysis")

In [ ]:
y=train[['toxic', 'severe_toxic']].values
train_sentence= train['comment_text']
test_sentence= test['comment_text']

Took lot of help from https://www.kaggle.com/sbongo/for-beginners-tackling-toxic-using-keras and https://www.kaggle.com/metrisrashmi/toxic-comments-problem-novice-s-approach.
***Please sure do comment. Any suggestion will be very handy.***